In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import AmazonGoogleBenchmark

benchmark = AmazonGoogleBenchmark(data_dir_path="../data/")
benchmark

22:53:20 INFO:Extracting Amazon-Google...
22:53:20 INFO:Reading Amazon-Google record_dict...
22:53:20 INFO:Reading Amazon-Google train.csv...
22:53:20 INFO:Reading Amazon-Google valid.csv...
22:53:20 INFO:Reading Amazon-Google test.csv...


<AmazonGoogleBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/amazon_google_exp_data.zip

## Preprocess

In [5]:
field_list = ['title', 'manufacturer', 'price']

In [6]:
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for record_dict in [benchmark.train_record_dict, benchmark.valid_record_dict, benchmark.test_record_dict]:
    for record in record_dict.values():
        for field in field_list:
            record[field] = clean_str(record[field])

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
from entity_embed import PairNumericalizer

pair_numericalizer = PairNumericalizer(field_list)

In [9]:
batch_size = 32
eval_batch_size = 256
datamodule = benchmark.build_matcher_datamodule(
    pair_numericalizer=pair_numericalizer,
    batch_size=batch_size,
    eval_batch_size=eval_batch_size,
    random_seed=random_seed
)

## Training

In [10]:
from entity_embed import Matcher

model = Matcher(
    pair_numericalizer=pair_numericalizer
)

22:53:24 INFO:Load pretrained SentenceTransformer: stsb-distilbert-base
22:53:24 INFO:Did not find folder stsb-distilbert-base
22:53:24 INFO:Search model on server: http://sbert.net/models/stsb-distilbert-base.zip
22:53:24 INFO:Load SentenceTransformer from folder: /home/fjsj/.cache/torch/sentence_transformers/sbert.net_models_stsb-distilbert-base
22:53:25 INFO:Use pytorch device: cuda


In [11]:
trainer = model.fit(
    datamodule,
    min_epochs=5,
    max_epochs=100,
    check_val_every_n_epoch=1,
    early_stop_monitor="valid_f1_at_0.5",
    tb_save_dir='../tb_logs',
    tb_name=f'matcher-{benchmark.dataset_name}'
)

22:53:25 INFO:GPU available: True, used: True
22:53:25 INFO:TPU available: False, using: 0 TPU cores
22:53:25 INFO:Train positive pair count: 699
22:53:25 INFO:Train negative pair count: 6175
22:53:25 INFO:Valid positive pair count: 234
22:53:25 INFO:Valid positive pair count: 2059
22:53:25 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
22:53:29 INFO:
  | Name        | Type              | Params
--------------------------------------------------
0 | matcher_net | MatcherNet        | 66.4 M
1 | loss_fn     | BCEWithLogitsLoss | 0     
--------------------------------------------------
66.4 M    Trainable params
0         Non-trainable params
66.4 M    Total params
265.455   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

23:12:35 INFO:Loading the best validation model from ../tb_logs/matcher-Amazon-Google/version_14/checkpoints/epoch=6-step=1504.ckpt...
23:12:35 INFO:Load pretrained SentenceTransformer: stsb-distilbert-base
23:12:35 INFO:Did not find folder stsb-distilbert-base
23:12:35 INFO:Search model on server: http://sbert.net/models/stsb-distilbert-base.zip
23:12:35 INFO:Load SentenceTransformer from folder: /home/fjsj/.cache/torch/sentence_transformers/sbert.net_models_stsb-distilbert-base
23:12:36 INFO:Use pytorch device: cuda


In [12]:
model.validate(datamodule)

{'valid_f1_at_0.3': tensor(0.7197),
 'valid_f1_at_0.5': tensor(0.7318),
 'valid_f1_at_0.7': tensor(0.7099),
 'valid_f1_at_0.9': tensor(0.6526),
 'valid_precision_at_0.3': tensor(0.6238),
 'valid_precision_at_0.5': tensor(0.6632),
 'valid_precision_at_0.7': tensor(0.6757),
 'valid_precision_at_0.9': tensor(0.7240),
 'valid_recall_at_0.3': tensor(0.8504),
 'valid_recall_at_0.5': tensor(0.8162),
 'valid_recall_at_0.7': tensor(0.7479),
 'valid_recall_at_0.9': tensor(0.5940)}

## Testing

In [13]:
model.test(datamodule)

23:13:16 INFO:Test positive pair count: 234
23:13:16 INFO:Test positive pair count: 2059


{'test_f1_at_0.3': tensor(0.7509),
 'test_f1_at_0.5': tensor(0.7514),
 'test_f1_at_0.7': tensor(0.7526),
 'test_f1_at_0.9': tensor(0.7067),
 'test_precision_at_0.3': tensor(0.6500),
 'test_precision_at_0.5': tensor(0.6842),
 'test_precision_at_0.7': tensor(0.7216),
 'test_precision_at_0.9': tensor(0.7688),
 'test_recall_at_0.3': tensor(0.8889),
 'test_recall_at_0.5': tensor(0.8333),
 'test_recall_at_0.7': tensor(0.7863),
 'test_recall_at_0.9': tensor(0.6538)}